### Crear Datasets (en forma de series temporales) de las variables que componen OASIS.

Este notebook prepara los Datasets en forma de series temporales, listos para entrenarlo con modelos predictivos: extraer y validar datos de las variables que componen OASIS, ordenarlos en series temporales, imputar los missing values y separar los datos en train/test sets.

In [1]:
import pandas as pd
import numpy as np
from tqdm import tqdm
import csv
import os
import sys

In [2]:
import shutil
import random
random.seed(42)

In [3]:
patients=pd.read_csv('/data/mimic/PATIENTS.csv')
print(patients.shape)
print('unique SUBJECT_ID:', patients.SUBJECT_ID.nunique())
patients.head()

(46520, 8)
unique SUBJECT_ID: 46520


,ROW_ID,SUBJECT_ID,GENDER,DOB,DOD,DOD_HOSP,DOD_SSN,EXPIRE_FLAG
0,234,249,F,2075-03-13 00:00:00,NaN,NaN,NaN,0
1,235,250,F,2164-12-27 00:00:00,2188-11-22 00:00:00,2188-11-22 00:00:00,NaN,1
2,236,251,M,2090-03-15 00:00:00,NaN,NaN,NaN,0
3,237,252,M,2078-03-06 00:00:00,NaN,NaN,NaN,0
4,238,253,F,2089-11-26 00:00:00,NaN,NaN,NaN,0


In [4]:
admissions=pd.read_csv('/data/mimic/ADMISSIONS.csv')
print(admissions.shape)
print('unique SUBJECT_ID:', admissions.SUBJECT_ID.nunique())
print('unique HADM_ID   :', admissions.HADM_ID.nunique())
admissions.head()

(58976, 19)
unique SUBJECT_ID: 46520
unique HADM_ID   : 58976


,ROW_ID,SUBJECT_ID,HADM_ID,ADMITTIME,DISCHTIME,DEATHTIME,ADMISSION_TYPE,ADMISSION_LOCATION,DISCHARGE_LOCATION,INSURANCE,LANGUAGE,RELIGION,MARITAL_STATUS,ETHNICITY,EDREGTIME,EDOUTTIME,DIAGNOSIS,HOSPITAL_EXPIRE_FLAG,HAS_CHARTEVENTS_DATA
0,21,22,165315,2196-04-09 12:26:00,2196-04-10 15:54:00,NaN,EMERGENCY,EMERGENCY ROOM ADMIT,DISC-TRAN CANCER/CHLDRN H,Private,NaN,UNOBTAINABLE,MARRIED,WHITE,2196-04-09 10:06:00,2196-04-09 13:24:00,BENZODIAZEPINE OVERDOSE,0,1
1,22,23,152223,2153-09-03 07:15:00,2153-09-08 19:10:00,NaN,ELECTIVE,PHYS REFERRAL/NORMAL DELI,HOME HEALTH CARE,Medicare,NaN,CATHOLIC,MARRIED,WHITE,NaN,NaN,CORONARY ARTERY DISEASE\CORONARY ARTERY BYPASS...,0,1
2,23,23,124321,2157-10-18 19:34:00,2157-10-25 14:00:00,NaN,EMERGENCY,TRANSFER FROM HOSP/EXTRAM,HOME HEALTH CARE,Medicare,ENGL,CATHOLIC,MARRIED,WHITE,NaN,NaN,BRAIN MASS,0,1
3,24,24,161859,2139-06-06 16:14:00,2139-06-09 12:48:00,NaN,EMERGENCY,TRANSFER FROM HOSP/EXTRAM,HOME,Private,NaN,PROTESTANT QUAKER,SINGLE,WHITE,NaN,NaN,INTERIOR MYOCARDIAL INFARCTION,0,1
4,25,25,129635,2160-11-02 02:06:00,2160-11-05 14:55:00,NaN,EMERGENCY,EMERGENCY ROOM ADMIT,HOME,Private,NaN,UNOBTAINABLE,MARRIED,WHITE,2160-11-02 01:01:00,2160-11-02 04:27:00,ACUTE CORONARY SYNDROME,0,1


In [5]:
admissions=admissions.drop(['ROW_ID', 'ADMISSION_LOCATION', 'DISCHARGE_LOCATION', 'LANGUAGE', 
                        'RELIGION', 'MARITAL_STATUS', 'EDREGTIME', 'EDOUTTIME'], axis=1)
admissions=pd.merge(patients, admissions, on=['SUBJECT_ID'])
print(admissions.shape)
print('unique SUBJECT_ID:', admissions.SUBJECT_ID.nunique())
print('unique HADM_ID   :', admissions.HADM_ID.nunique())
admissions.head()

(58976, 18)
unique SUBJECT_ID: 46520
unique HADM_ID   : 58976


,ROW_ID,SUBJECT_ID,GENDER,DOB,DOD,DOD_HOSP,DOD_SSN,EXPIRE_FLAG,HADM_ID,ADMITTIME,DISCHTIME,DEATHTIME,ADMISSION_TYPE,INSURANCE,ETHNICITY,DIAGNOSIS,HOSPITAL_EXPIRE_FLAG,HAS_CHARTEVENTS_DATA
0,234,249,F,2075-03-13 00:00:00,NaN,NaN,NaN,0,116935,2149-12-17 20:41:00,2149-12-31 14:55:00,NaN,EMERGENCY,Medicare,WHITE,UNSTABLE ANGINA;ASTHMA;BRONCHITIS,0,1
1,234,249,F,2075-03-13 00:00:00,NaN,NaN,NaN,0,149546,2155-02-03 20:16:00,2155-02-14 11:15:00,NaN,EMERGENCY,Medicare,WHITE,GI BLEED/ CHEST PAIN,0,1
2,234,249,F,2075-03-13 00:00:00,NaN,NaN,NaN,0,158975,2156-04-27 15:33:00,2156-05-14 15:30:00,NaN,EMERGENCY,Medicare,WHITE,GI BLEEDING\COLONOSCOPY,0,1
3,235,250,F,2164-12-27 00:00:00,2188-11-22 00:00:00,2188-11-22 00:00:00,NaN,1,124271,2188-11-12 09:22:00,2188-11-22 12:00:00,2188-11-22 12:00:00,EMERGENCY,Self Pay,BLACK/AFRICAN AMERICAN,PNEUMONIA;R/O TB,1,1
4,236,251,M,2090-03-15 00:00:00,NaN,NaN,NaN,0,117937,2110-07-27 06:46:00,2110-07-29 15:23:00,NaN,EMERGENCY,Private,UNKNOWN/NOT SPECIFIED,INTRACRANIAL HEAD BLEED,0,1


In [6]:
icustays=pd.read_csv('/data/mimic/ICUSTAYS.csv')
print(icustays.shape)
print('unique SUBJECT_ID:', icustays.SUBJECT_ID.nunique())
print('unique HADM_ID   :', icustays.HADM_ID.nunique())
print('unique ICUSTAY_ID:', icustays.ICUSTAY_ID.nunique())
icustays.head()

(61532, 12)
unique SUBJECT_ID: 46476
unique HADM_ID   : 57786
unique ICUSTAY_ID: 61532


,ROW_ID,SUBJECT_ID,HADM_ID,ICUSTAY_ID,DBSOURCE,FIRST_CAREUNIT,LAST_CAREUNIT,FIRST_WARDID,LAST_WARDID,INTIME,OUTTIME,LOS
0,365,268,110404,280836,carevue,MICU,MICU,52,52,2198-02-14 23:27:38,2198-02-18 05:26:11,3.2490
1,366,269,106296,206613,carevue,MICU,MICU,52,52,2170-11-05 11:05:29,2170-11-08 17:46:57,3.2788
2,367,270,188028,220345,carevue,CCU,CCU,57,57,2128-06-24 15:05:20,2128-06-27 12:32:29,2.8939
3,368,271,173727,249196,carevue,MICU,SICU,52,23,2120-08-07 23:12:42,2120-08-10 00:39:04,2.0600
4,369,272,164716,210407,carevue,CCU,CCU,57,57,2186-12-25 21:08:04,2186-12-27 12:01:13,1.6202


In [7]:
icustays=icustays.drop(['ROW_ID', 'LAST_CAREUNIT', 'LAST_WARDID'], axis=1)
admissions=pd.merge(admissions, icustays, on=['SUBJECT_ID', 'HADM_ID'])
print(admissions.shape)
print('unique SUBJECT_ID:', admissions.SUBJECT_ID.nunique())
print('unique HADM_ID   :', admissions.HADM_ID.nunique())
print('unique ICUSTAY_ID:', admissions.ICUSTAY_ID.nunique())
admissions.head()

(61532, 25)
unique SUBJECT_ID: 46476
unique HADM_ID   : 57786
unique ICUSTAY_ID: 61532


,ROW_ID,SUBJECT_ID,GENDER,DOB,DOD,DOD_HOSP,DOD_SSN,EXPIRE_FLAG,HADM_ID,ADMITTIME,...,DIAGNOSIS,HOSPITAL_EXPIRE_FLAG,HAS_CHARTEVENTS_DATA,ICUSTAY_ID,DBSOURCE,FIRST_CAREUNIT,FIRST_WARDID,INTIME,OUTTIME,LOS
0,234,249,F,2075-03-13 00:00:00,NaN,NaN,NaN,0,116935,2149-12-17 20:41:00,...,UNSTABLE ANGINA;ASTHMA;BRONCHITIS,0,1,215044,carevue,TSICU,23,2149-12-18 20:06:02,2149-12-24 13:31:45,5.7262
1,234,249,F,2075-03-13 00:00:00,NaN,NaN,NaN,0,149546,2155-02-03 20:16:00,...,GI BLEED/ CHEST PAIN,0,1,269035,metavision,MICU,50,2155-02-03 20:17:29,2155-02-05 18:34:02,1.9282
2,234,249,F,2075-03-13 00:00:00,NaN,NaN,NaN,0,149546,2155-02-03 20:16:00,...,GI BLEED/ CHEST PAIN,0,1,263055,metavision,TSICU,14,2155-02-07 18:51:16,2155-02-11 16:00:39,3.8815
3,234,249,F,2075-03-13 00:00:00,NaN,NaN,NaN,0,158975,2156-04-27 15:33:00,...,GI BLEEDING\COLONOSCOPY,0,1,282599,metavision,MICU,52,2156-05-01 18:10:12,2156-05-03 18:43:45,2.0233
4,234,249,F,2075-03-13 00:00:00,NaN,NaN,NaN,0,158975,2156-04-27 15:33:00,...,GI BLEEDING\COLONOSCOPY,0,1,263882,metavision,MICU,52,2156-05-10 17:47:35,2156-05-11 19:16:03,1.0614


In [8]:
#Drop unnecessary columns
admissions=admissions.drop(['INSURANCE','ETHNICITY'], axis=1)
admissions=admissions.drop(['DOD_HOSP', 'DOD_SSN', 'EXPIRE_FLAG','DIAGNOSIS','HOSPITAL_EXPIRE_FLAG','HAS_CHARTEVENTS_DATA','FIRST_CAREUNIT','FIRST_WARDID'], axis=1)
admissions=admissions.drop(['ROW_ID','GENDER','DBSOURCE'], axis=1)

In [9]:
#we compute the age at the time the patient is admitted to the ICU from date of birth dob, then we drop dob.
admissions['DOB'] = pd.to_datetime(admissions['DOB']).dt.date
admissions['INTIME_TMP'] = pd.to_datetime(admissions['INTIME']).dt.date
admissions['AGE'] = admissions.apply(lambda e: (e['INTIME_TMP'] - e['DOB']).days/365, axis=1)
admissions=admissions.drop(['DOB'], axis=1)
admissions=admissions.drop(['INTIME_TMP'], axis=1)

In [10]:
#Excluir admissions < 18 years old, ya que tendran medidas fisiológicas distintas de las de los adultos 
admissions = admissions[~(admissions['AGE'] < 18)]
print(admissions.shape)
print('unique SUBJECT_ID:', admissions.SUBJECT_ID.nunique())
print('unique HADM_ID   :', admissions.HADM_ID.nunique())
print('unique ICUSTAY_ID:', admissions.ICUSTAY_ID.nunique())

(53332, 12)
unique SUBJECT_ID: 38512
unique HADM_ID   : 49695
unique ICUSTAY_ID: 53332


In [11]:
#we compute the  Pre-ICU length of stay in hours. 
prelos_tmp= pd.to_datetime(admissions['INTIME'])-pd.to_datetime(admissions['ADMITTIME'])
admissions['PRELOS']=np.round(prelos_tmp.dt.total_seconds()/3600, 1)
admissions['PRELOS']=np.where(admissions['PRELOS']<0,0,admissions['PRELOS'])
#OBS:hours, (hay filas que salen negativos, error de introducir admittime y intime)

In [12]:
older89=len(admissions[(admissions['AGE']>=300)])
older89

2721

In [13]:
admissions['AGE']=admissions['AGE'].apply(lambda x: np.random.triangular(89.01, 89.01, 100) if x >= 300 else x)
admissions.shape

(53332, 13)

In [14]:
admissions['DOD'] = pd.to_datetime(admissions['DOD'])
admissions['ADMITTIME'] = pd.to_datetime(admissions['ADMITTIME'])
admissions['DISCHTIME'] = pd.to_datetime(admissions['DISCHTIME'])
admissions['DEATHTIME'] = pd.to_datetime(admissions['DEATHTIME'])
admissions['MORTALITY_INHOSPITAL'] = admissions['DOD'].notnull() & ((admissions['ADMITTIME'] <= admissions['DOD']) & (admissions['DISCHTIME'] >= admissions['DOD']))
admissions['MORTALITY_INHOSPITAL'] = admissions['MORTALITY_INHOSPITAL'] | (admissions['DEATHTIME'].notnull() & ((admissions['ADMITTIME'] <= admissions['DEATHTIME']) & (admissions['DISCHTIME'] >= admissions['DEATHTIME'])))
admissions['MORTALITY_INHOSPITAL'] = admissions['MORTALITY_INHOSPITAL'].astype(int)

In [15]:
admissions['MORTALITY_INHOSPITAL'].value_counts()

0    46816
1     6516
Name: MORTALITY_INHOSPITAL, dtype: int64

In [16]:
#admissions['INTIME'] = pd.to_datetime(admissions['INTIME'])
#admissions['OUTTIME'] = pd.to_datetime(admissions['OUTTIME'])
#admissions['MORTALITY_INUNIT'] = admissions['DOD'].notnull() & ((admissions['INTIME'] <= admissions['DOD']) & (admissions['OUTTIME'] >= admissions['DOD']))
#admissions['MORTALITY_INUNIT'] = admissions['MORTALITY_INUNIT'] | (admissions['DEATHTIME'].notnull() & ((admissions['INTIME'] <= admissions['DEATHTIME']) & (admissions['OUTTIME'] >= admissions['DEATHTIME'])))
#admissions['MORTALITY_INUNIT'] = admissions['MORTALITY_INUNIT'].astype(int)
#admissions['MORTALITY_INUNIT'].value_counts()


In [17]:
admissions.head(5)

,SUBJECT_ID,DOD,HADM_ID,ADMITTIME,DISCHTIME,DEATHTIME,ADMISSION_TYPE,ICUSTAY_ID,INTIME,OUTTIME,LOS,AGE,PRELOS,MORTALITY_INHOSPITAL
0,249,NaT,116935,2149-12-17 20:41:00,2149-12-31 14:55:00,NaT,EMERGENCY,215044,2149-12-18 20:06:02,2149-12-24 13:31:45,5.7262,74.816438,23.4,0
1,249,NaT,149546,2155-02-03 20:16:00,2155-02-14 11:15:00,NaT,EMERGENCY,269035,2155-02-03 20:17:29,2155-02-05 18:34:02,1.9282,79.947945,0.0,0
2,249,NaT,149546,2155-02-03 20:16:00,2155-02-14 11:15:00,NaT,EMERGENCY,263055,2155-02-07 18:51:16,2155-02-11 16:00:39,3.8815,79.958904,94.6,0
3,249,NaT,158975,2156-04-27 15:33:00,2156-05-14 15:30:00,NaT,EMERGENCY,282599,2156-05-01 18:10:12,2156-05-03 18:43:45,2.0233,81.189041,98.6,0
4,249,NaT,158975,2156-04-27 15:33:00,2156-05-14 15:30:00,NaT,EMERGENCY,263882,2156-05-10 17:47:35,2156-05-11 19:16:03,1.0614,81.213699,314.2,0


In [18]:
admissions.isnull().sum()

SUBJECT_ID                  0
DOD                     29223
HADM_ID                     0
ADMITTIME                   0
DISCHTIME                   0
DEATHTIME               46797
ADMISSION_TYPE              0
ICUSTAY_ID                  0
INTIME                      0
OUTTIME                     3
LOS                         3
AGE                         0
PRELOS                      0
MORTALITY_INHOSPITAL        0
dtype: int64

In [19]:
output_path = 'timesSeriesv2_cohorte/data/root/'
try:
    os.makedirs(output_path)
except:
    pass

In [20]:
#Store all valid icustays admissions in all_stays.csv file.
admissions.to_csv(os.path.join(output_path, 'all_stays.csv'), index=False)

###  Separar cada icustay en carpetas distintas (split and conquer)

In [21]:
def break_up_stays(stays, output_path):
    nb_stays = stays.shape[0]
    stays_ids = stays.ICUSTAY_ID.unique()
    for icustay_id in tqdm(stays_ids, total=nb_stays, desc='Breaking up stays'):
        dn = os.path.join(output_path, str(icustay_id))
        try:
            os.makedirs(dn)
        except:
            pass

        stays[stays.ICUSTAY_ID == icustay_id].to_csv(os.path.join(dn, 'stays.csv'),index=False)


In [22]:
break_up_stays(admissions, output_path)

Breaking up stays: 100%|█████████████| 53332/53332 [02:41<00:00, 330.19it/s]


### Extraer las mediciones que nos interesan (temperatura, frecuencia respiratoria, frecuencia cardíaca ...) y distribuirlas por las carpetas(icustay) correspondientes.

In [23]:
"""
https://github.com/MIT-LCP/mimic-code/blob/master/concepts/firstday/gcs_first_day.sql

723  Verbal Response
454  Motor Response
184  Eye Opening
198  GCS Total

223900 GCS - Verbal Response
223901 GCS - Motor Response
220739 GCS - Eye Opening

https://github.com/MIT-LCP/mimic-code/blob/master/concepts/cookbook/temp.sql
676 -- Temperature C
677 -- Temperature C (calc)
678 -- Temperature F
679 -- Temperature F (calc)
223761 -- Temperature Fahrenheit
223762 -- Temperature Celsius


#TODO: review ventilation itemIDs
https://github.com/MIT-LCP/mimic-code/blob/master/concepts/durations/ventilation_classification.sql
¿¿226260 Mechanically Ventilated?? 
Si en chartevents.csv hay 720, 223849 -- Ventilator Mode, 223848 -- Ventilator Type
indica que ese ICUSTAY_ID fue ventilated?
Actual:
225792	Invasive Ventilation
225794	Non-invasive Ventilation




https://github.com/MIT-LCP/mimic-code/blob/master/concepts/firstday/vitals_first_day.sql

220045,211  HeartRate

MAP:
456	NBP Mean
52 Arterial BP Mean
6702 Arterial BP Mean #2
443 Manual BP Mean(calc)
220052 Arterial Blood Pressure mean
220181 Non Invasive Blood Pressure mean
225312 ART BP mean

RespRate:
615 Resp Rate (Total)
618 Respiratory Rate
220210 Respiratory Rate
224690 Respiratory Rate (Total)



https://github.com/MIT-LCP/mimic-code/blob/master/concepts/firstday/urine_output_first_day.sql
#TODO: revisar? ponerlos todos?
urine(todo en mL):
-- these are the most frequently occurring urine output observations in CareVue
40055, -- "Urine Out Foley"
43175, -- "Urine ."  <--- no uso
40069, -- "Urine Out Void"
40094, -- "Urine Out Condom Cath"
40715, -- "Urine Out Suprapubic"
40473, -- "Urine Out IleoConduit"
40085, -- "Urine Out Incontinent"
40057, -- "Urine Out Rt Nephrostomy"
40056, -- "Urine Out Lt Nephrostomy"
40405, -- "Urine Out Other"
40428, -- "Urine Out Straight Cath"
40086,--   Urine Out Incontinent
40096, -- "Urine Out Ureteral Stent #1"
40651, -- "Urine Out Ureteral Stent #2"

-- these are the most frequently occurring urine output observations in MetaVision
226559, -- "Foley"
226560, -- "Void"
226561, -- "Condom Cath"
226584, -- "Ileoconduit"
226563, -- "Suprapubic"
226564, -- "R Nephrostomy"
226565, -- "L Nephrostomy"
226567, --  Straight Cath
226557, -- R Ureteral Stent
226558, -- L Ureteral Stent

"""

'\nhttps://github.com/MIT-LCP/mimic-code/blob/master/concepts/firstday/gcs_first_day.sql\n\n723  Verbal Response\n454  Motor Response\n184  Eye Opening\n198  GCS Total\n\n223900 GCS - Verbal Response\n223901 GCS - Motor Response\n220739 GCS - Eye Opening\n\nhttps://github.com/MIT-LCP/mimic-code/blob/master/concepts/cookbook/temp.sql\n676 -- Temperature C\n677 -- Temperature C (calc)\n678 -- Temperature F\n679 -- Temperature F (calc)\n223761 -- Temperature Fahrenheit\n223762 -- Temperature Celsius\n\n\n#TODO: review ventilation itemIDs\nhttps://github.com/MIT-LCP/mimic-code/blob/master/concepts/durations/ventilation_classification.sql\n¿¿226260 Mechanically Ventilated?? \nSi en chartevents.csv hay 720, 223849 -- Ventilator Mode, 223848 -- Ventilator Type\nindica que ese ICUSTAY_ID fue ventilated?\nActual:\n225792\tInvasive Ventilation\n225794\tNon-invasive Ventilation\n\n\n\n\nhttps://github.com/MIT-LCP/mimic-code/blob/master/concepts/firstday/vitals_first_day.sql\n\n220045,211  HeartRa

In [24]:
itemids = [223762,223761,676,677,678,679,220045,211,220210,615,618,220052,220181,225312,456,52,6702,443,220739,184,223901,454,223900,723,198,225792,225794,40055,226559,40069,226560,40094,40715,40473,40085,40057,40056,40405,40428,40086,40096,40651,226561,226584,226563,226564,226565,226567,226557,226558]

In [25]:
def read_events_table_by_row(mimic3_path, table):
    nb_rows = {'chartevents': 330712484,  'outputevents': 4349219, 'procedureevents_mv': 258067,}
    reader = csv.DictReader(open(os.path.join(mimic3_path, table.upper() + '.csv'), 'r'))
    for i, row in enumerate(reader):
        if 'ICUSTAY_ID' not in row:
            row['ICUSTAY_ID'] = ''
        yield row, i, nb_rows[table.lower()]


In [26]:
def read_events_table_and_break_up_by_stays(mimic3_path, table, output_path,
                                              items_to_keep=None, stays_to_keep=None):
    
    if(table == 'PROCEDUREEVENTS_MV'):
        obs_header = ['SUBJECT_ID', 'HADM_ID', 'ICUSTAY_ID', 'STARTTIME', 'ITEMID']
    
    elif(table == 'OUTPUTEVENTS'):
        obs_header = ['SUBJECT_ID', 'HADM_ID', 'ICUSTAY_ID', 'CHARTTIME', 'ITEMID', 'VALUE','VALUEUOM','ISERROR']
    
    else:
        obs_header = ['SUBJECT_ID', 'HADM_ID', 'ICUSTAY_ID', 'CHARTTIME', 'ITEMID', 'VALUE', 'VALUENUM','VALUEUOM','ERROR']

    
    if items_to_keep is not None:
        items_to_keep = set([str(s) for s in items_to_keep])

    if stays_to_keep is not None:
        stays_to_keep = set([str(s) for s in stays_to_keep])
        
    class DataStats(object):
        def __init__(self):
            self.curr_icustay_id = ''
            self.curr_obs = []

    data_stats = DataStats()

    def write_current_observations():
        dn = os.path.join(output_path, str(data_stats.curr_icustay_id))
        try:
            os.makedirs(dn)
        except:
            pass
        
        if(table == 'PROCEDUREEVENTS_MV'):
            fn = os.path.join(dn, 'ventilation.csv')
        elif(table == 'OUTPUTEVENTS'):
            fn = os.path.join(dn, 'urine.csv')
        else:
            fn = os.path.join(dn, 'events.csv')
        
        
        if not os.path.exists(fn) or not os.path.isfile(fn):
            f = open(fn, 'w')
            f.write(','.join(obs_header) + '\n')
            f.close()
        w = csv.DictWriter(open(fn, 'a'), fieldnames=obs_header, quoting=csv.QUOTE_MINIMAL)
        w.writerows(data_stats.curr_obs)
        data_stats.curr_obs = []

    nb_rows_dict = {'chartevents': 330712484, 'outputevents': 4349219, 'procedureevents_mv': 258067}
    nb_rows = nb_rows_dict[table.lower()]

    for row, row_no, _ in tqdm(read_events_table_by_row(mimic3_path, table), total=nb_rows,
                                                        desc='Processing {} table'.format(table)):

        if (stays_to_keep is not None) and (row['ICUSTAY_ID'] not in stays_to_keep):
            continue
        if (items_to_keep is not None) and (row['ITEMID'] not in items_to_keep):
            continue        
        
        if(table == 'PROCEDUREEVENTS_MV'):
            row_out = {'SUBJECT_ID': row['SUBJECT_ID'],
                       'HADM_ID': row['HADM_ID'],
                       'ICUSTAY_ID': '' if 'ICUSTAY_ID' not in row else row['ICUSTAY_ID'],
                       'STARTTIME': row['STARTTIME'],
                       'ITEMID': row['ITEMID']}
        
        elif(table == 'OUTPUTEVENTS'):
            row_out = {'SUBJECT_ID': row['SUBJECT_ID'],
                       'HADM_ID': row['HADM_ID'],
                       'ICUSTAY_ID': '' if 'ICUSTAY_ID' not in row else row['ICUSTAY_ID'],
                       'CHARTTIME': row['CHARTTIME'],
                       'ITEMID': row['ITEMID'],
                       'VALUE': row['VALUE'],
                       'VALUEUOM': row['VALUEUOM'],
                        'ISERROR': row['ISERROR']}
        
        else:      
            row_out = {'SUBJECT_ID': row['SUBJECT_ID'],
                       'HADM_ID': row['HADM_ID'],
                       'ICUSTAY_ID': '' if 'ICUSTAY_ID' not in row else row['ICUSTAY_ID'],
                       'CHARTTIME': row['CHARTTIME'],
                       'ITEMID': row['ITEMID'],
                       'VALUE': row['VALUE'],
                       'VALUENUM': row['VALUENUM'],
                       'VALUEUOM': row['VALUEUOM'],
                       'ERROR': row['ERROR']}
            
        if data_stats.curr_icustay_id != '' and data_stats.curr_icustay_id != row['ICUSTAY_ID']:
            write_current_observations()
        data_stats.curr_obs.append(row_out)
        data_stats.curr_icustay_id = row['ICUSTAY_ID']

    if data_stats.curr_icustay_id != '':
        write_current_observations()

In [29]:
event_tables = ['PROCEDUREEVENTS_MV','OUTPUTEVENTS','CHARTEVENTS']
mimic3_path = '/data/mimic/'

In [30]:
#Esta celda tarda mucho, 1h aprox 
#Ejecutar solo una vez
items_to_keep = set([int(itemid) for itemid in itemids]) 
for table in event_tables:
    read_events_table_and_break_up_by_stays(mimic3_path, table, output_path, items_to_keep=items_to_keep,stays_to_keep=admissions.ICUSTAY_ID.unique())

Processing PROCEDUREEVENTS_MV table: 100%|▉| 258066/258067 [00:06<00:00, 395
Processing OUTPUTEVENTS table: 100%|▉| 4349218/4349219 [04:26<00:00, 16302.4
Processing CHARTEVENTS table: 100%|▉| 330712483/330712484 [1:08:20<00:00, 80


### Crear timeSeries.csv (sin incluir mediciones: ventilation,urine) para cada icustay

In [31]:
stays_root_path = 'timesSeriesv2_cohorte/data/root/'

In [32]:
def read_stays(subject_path):
    stays = pd.read_csv(os.path.join(subject_path, 'stays.csv'))
    return stays

def read_ventilation(subject_path):
    fn = os.path.join(subject_path, 'ventilation.csv')
    if not os.path.exists(fn) or not os.path.isfile(fn):
        ventilations = pd.DataFrame(columns=['SUBJECT_ID','HADM_ID','ICUSTAY_ID','STARTTIME'])         
    else:
        ventilations = pd.read_csv(fn)
    return ventilations

def read_urine(subject_path):
    fn = os.path.join(subject_path, 'urine.csv')
    if not os.path.exists(fn) or not os.path.isfile(fn):
        urine = pd.DataFrame(columns=['SUBJECT_ID','HADM_ID','ICUSTAY_ID','URINE_TIME','URINE']) #create empty dataframe with columns name URINE    
    else:
        urine = pd.read_csv(fn)
        urine=urine[urine['ISERROR'].isnull()]
        urine=urine.rename({'CHARTTIME':'URINE_TIME', 'VALUE':'URINE'}, axis='columns')
        urine = urine.drop(['ITEMID','VALUEUOM'],axis=1)

    return urine

In [33]:
def read_events(subject_path):
    fn = os.path.join(subject_path, 'events.csv')    
    if not os.path.exists(fn) or not os.path.isfile(fn):
        events = pd.DataFrame(columns=['SUBJECT_ID', 'HADM_ID', 'ICUSTAY_ID', 'CHARTTIME', 'ITEMID','VARIABLE','VALUE'])
    else:
        events = pd.read_csv(fn)
        events.CHARTTIME = pd.to_datetime(events.CHARTTIME)
        events = events[events.VALUE.notnull()]
    
        #hay events que tiene todo VALUE null, por ejemplo: 222558 
        if(events.shape[0] == 0):
            events =  pd.DataFrame(columns=['SUBJECT_ID', 'HADM_ID', 'ICUSTAY_ID', 'CHARTTIME', 'ITEMID','VARIABLE','VALUE'])
        else:
        
            #filtrar filas con error
            events = events.loc[(events['ERROR'] == 0.0) | (events['ERROR'].isna()) ]
            events = events.drop(['ERROR'],axis=1)

            #hay events que todos con error
            if(events.shape[0] == 0):
                events =  pd.DataFrame(columns=['SUBJECT_ID', 'HADM_ID', 'ICUSTAY_ID', 'CHARTTIME', 'ITEMID','VARIABLE','VALUE'])
            else:
                
                #convert fahrenheit to celsius
                events.loc[events['ITEMID'].isin([223761,678,679]),'VALUE'] = (pd.to_numeric(events.loc[events['ITEMID'].isin([223761,678,679])]['VALUE'])- 32) * 5 / 9

                #assign variable name to itemid 
                events.loc[events['ITEMID'].isin([223762,676,677,223761,678,679]), 'VARIABLE'] = 'TEMP_C'
                events.loc[events['ITEMID'].isin([220045,211]), 'VARIABLE'] = 'HRATE'
                events.loc[events['ITEMID'].isin([220210,224690,615,618]), 'VARIABLE'] = 'RESP_RATE'
                events.loc[events['ITEMID'].isin([220052,220181,225312,456,52,6702,443]), 'VARIABLE'] = 'MAP'



                events.loc[events['ITEMID'].isin([220739,184]), 'VARIABLE'] = 'gcs_e'
                events.loc[events['ITEMID'].isin([223901,454]), 'VARIABLE'] = 'gcs_m'
                events.loc[events['ITEMID'].isin([223900,723]), 'VARIABLE'] = 'gcs_v'
                events.loc[events['ITEMID'] == 198, 'VARIABLE'] = 'gcs_total_carevue'

                #para gcs score coger la columna VALUENUM, que contiene valores enteros
                events.loc[events['ITEMID'].isin([220739,184,223901,454,223900,723,198]),'VALUE'] = events.loc[events['ITEMID'].isin([220739,184,223901,454,223900,723,198])]['VALUENUM']


                events = events.drop(['VALUENUM','VALUEUOM'],axis=1)

        
    return events


In [34]:
#recoger datos estaticos de cada icustay: como LOS, PRELOS, AGE, ADMISSION TYPE, MORTALITY_INHOSPITAL 
def assemble_episodic_data(stays):
    data = {'Icustay': stays.ICUSTAY_ID,'Age': stays.AGE,'Length of Stay': stays.LOS,
            'PreICU LOS': stays.PRELOS,'Admission Type': stays.ADMISSION_TYPE,
           'Inhospital Mortality': stays.MORTALITY_INHOSPITAL}
    data = pd.DataFrame(data).set_index('Icustay')
    data = data[['Age', 'Length of Stay','PreICU LOS','Admission Type','Inhospital Mortality' ]]
    return data


In [35]:
def get_events_for_stay(events, icustayid):
    idx = (events.ICUSTAY_ID == icustayid)
    events = events[idx]
    del events['ICUSTAY_ID']
    return events

In [36]:
def add_hours_elpased_to_events(events, intime, remove_charttime=True):
    events = events.copy()
    if events.shape[0] != 0:   
        hours_tmp = events.CHARTTIME - intime
        events['HOURS'] = np.round(hours_tmp.dt.total_seconds()/3600, 0)    
    else:
        events['HOURS'] = 0 #esta hora para AGE, PRELOS (que siempre estan)
    if remove_charttime:
        del events['CHARTTIME']
    return events

In [37]:
def convert_events_to_timeseries(events, variable_column='VARIABLE', variables=[]):
    metadata = events[['CHARTTIME', 'ICUSTAY_ID']].sort_values(by=['CHARTTIME', 'ICUSTAY_ID'])\
                    .drop_duplicates(keep='first').set_index('CHARTTIME')

    timeseries = events[['CHARTTIME', variable_column, 'VALUE']]\
                    .sort_values(by=['CHARTTIME', variable_column, 'VALUE'], axis=0).drop_duplicates(subset=['CHARTTIME', variable_column], keep='last')
        
    timeseries = timeseries.pivot(index='CHARTTIME', columns=variable_column, values='VALUE')\
                    .merge(metadata, left_index=True, right_index=True)\
                    .sort_index(axis=0).reset_index()
    for v in variables:
        if v not in timeseries:
            timeseries[v] = np.nan
    return timeseries

In [38]:
variables = ['HRATE','MAP','RESP_RATE','TEMP_C','gcs_e','gcs_m','gcs_v','gcs_total_carevue'] 

In [39]:
#Execution time: 1 h 30 min

#for stay_dir in tqdm(['239511','212355'], desc='Iterating over stays'): #prueba
for stay_dir in tqdm(os.listdir(stays_root_path), desc='Iterating over stays'):
    dn = os.path.join(stays_root_path, stay_dir)
    try:
        icustay_id = int(stay_dir)
        if not os.path.isdir(dn):
            raise Exception
    except:
        continue
        

    try:
        # reading tables of this icustay_id
        stays = read_stays(os.path.join(stays_root_path, stay_dir))
        events = read_events(os.path.join(stays_root_path, stay_dir))
        #df_ventilated = read_ventilation(os.path.join(stays_root_path, stay_dir))
        #df_urine = read_urine(os.path.join(stays_root_path, stay_dir))
    
    except:
        sys.stderr.write('Error reading from disk for icustay_id: {}\n'.format(icustay_id))
        continue
    
    episodic_data = assemble_episodic_data(stays)
    
    # converting to time series
    if events.shape[0] == 0:
        print("(no valid events for icustay_id: {}\n".format(icustay_id))
    timeseries = convert_events_to_timeseries(events, variables=variables)

# extracting separate episodes (icustays) 
    for i in range(stays.shape[0]):
        #imprimir = False
        stay_id = stays.ICUSTAY_ID.iloc[i]
        intime = pd.to_datetime(stays.INTIME.iloc[i])
        episode = get_events_for_stay(timeseries, stay_id)
        # converting to time series
        
        if episode.shape[0] == 0:
            print("(no data for this episode {} of icustay_id: {}\n".format(i+1, icustay_id))

        episode = add_hours_elpased_to_events(episode, intime).set_index('HOURS').sort_index(axis=0)
        episodic_data.loc[episodic_data.index == stay_id].to_csv(os.path.join(stays_root_path,stay_dir,'episode{}.csv'.format(i+1)),index_label='Icustay')
        
        columns = list(episode.columns)
        columns_sorted = sorted(columns, key=(lambda x: "" if x == "HOURS" else x))
        episode = episode[columns_sorted]
        if episode.shape[0] == 0:
            episode['AGE'] = stays.AGE
            episode['PRELOS'] = stays.PRELOS
        else:
            episode['AGE'] = stays.AGE[0]
            episode['PRELOS'] = stays.PRELOS[0]

        episode.to_csv(os.path.join(stays_root_path, stay_dir, 'episode{}_timeseries.csv'.format(i+1)),
                   index_label='Hours')
    

Iterating over stays:   0%|            | 73/53333 [00:10<1:55:27,  7.69it/s]

(no valid events for icustay_id: 253219

(no data for this episode 1 of icustay_id: 253219



Iterating over stays:   0%|           | 122/53333 [00:16<1:53:55,  7.78it/s]

(no valid events for icustay_id: 226586

(no data for this episode 1 of icustay_id: 226586



Iterating over stays:   0%|           | 185/53333 [00:29<1:33:26,  9.48it/s]

...

Iterating over stays: 100%|████████▉| 53267/53333 [2:18:27<00:59,  1.10it/s]

(no valid events for icustay_id: 233440

(no data for this episode 1 of icustay_id: 233440



Iterating over stays: 100%|████████▉| 53304/53333 [2:18:33<00:03,  9.60it/s]

(no valid events for icustay_id: 241818

(no data for this episode 1 of icustay_id: 241818



Iterating over stays: 100%|█████████| 53333/53333 [2:18:37<00:00,  6.41it/s]


In [40]:
#pd.read_csv('timesSeriesv2_cohorte/data/root/239511/stays.csv')

In [41]:
prueba_v3 = pd.read_csv('timesSeriesv2_cohorte/data/root/239511/episode1_timeseries.csv')
prueba_v3

,Hours,HRATE,MAP,RESP_RATE,TEMP_C,gcs_e,gcs_m,gcs_total_carevue,gcs_v,AGE,PRELOS
0,40.0,60,108.000000,16,NaN,NaN,NaN,NaN,NaN,63.139726,0.0
1,41.0,66,109.000000,22,NaN,NaN,NaN,NaN,NaN,63.139726,0.0
2,42.0,73,115.000000,20,NaN,NaN,NaN,NaN,NaN,63.139726,0.0
3,43.0,67,103.667000,24,NaN,NaN,NaN,NaN,NaN,63.139726,0.0
4,44.0,63,91.000000,22,36.611099,4.0,6.0,15.0,5.0,63.139726,0.0
5,45.0,66,96.000000,20,NaN,NaN,NaN,NaN,NaN,63.139726,0.0
6,46.0,57,95.666702,16,NaN,NaN,NaN,NaN,NaN,63.139726,0.0
7,47.0,53,86.666702,19,NaN,NaN,NaN,NaN,NaN,63.139726,0.0
8,48.0,60,102.000000,12,37.055599,4.0,6.0,15.0,5.0,63.139726,0.0
9,49.0,64,100.667000,20,NaN,NaN,NaN,NaN,NaN,63.139726,0.0


In [42]:
prueba_v4 = pd.read_csv('timesSeriesv2_cohorte/data/root/212355/episode1_timeseries.csv')
prueba_v4

,Hours,HRATE,MAP,RESP_RATE,TEMP_C,gcs_e,gcs_m,gcs_total_carevue,gcs_v,AGE,PRELOS
0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,46.487671,0.0


## Split train and test dataset según la tarea. 

Normalmente basta con train_test_split para tener conjunto de entrenamiento (Train set) y conjunto de prueba (Test set). Pero como queremos comparar con otros modelos (por ejemplo, con los de benchmark. Además, en el benchmark cada tarea tiene su cohorte específica), recomienda mantener un mismo Test set. Por lo tanto, usamos ficheros llamados 'listfile' para poder rastrear el Test/Train set. (obs. estas listfiles se obtienen ejecutando create_losxdays_icustayidListfiles.py y create_in_hospital_mortality_icustayidListFile.py)

### TimeSeries For LOS > 3 task:

In [43]:
stays_root_path = "/data/codi/OASIS/timesSeriesv2_cohorte/data/root"

In [44]:
cohorte_general = pd.read_csv(os.path.join(stays_root_path, "all_stays.csv"))
cohorte_general.head()

,SUBJECT_ID,DOD,HADM_ID,ADMITTIME,DISCHTIME,DEATHTIME,ADMISSION_TYPE,ICUSTAY_ID,INTIME,OUTTIME,LOS,AGE,PRELOS,MORTALITY_INHOSPITAL
0,249,NaN,116935,2149-12-17 20:41:00,2149-12-31 14:55:00,NaN,EMERGENCY,215044,2149-12-18 20:06:02,2149-12-24 13:31:45,5.7262,74.816438,23.4,0
1,249,NaN,149546,2155-02-03 20:16:00,2155-02-14 11:15:00,NaN,EMERGENCY,269035,2155-02-03 20:17:29,2155-02-05 18:34:02,1.9282,79.947945,0.0,0
2,249,NaN,149546,2155-02-03 20:16:00,2155-02-14 11:15:00,NaN,EMERGENCY,263055,2155-02-07 18:51:16,2155-02-11 16:00:39,3.8815,79.958904,94.6,0
3,249,NaN,158975,2156-04-27 15:33:00,2156-05-14 15:30:00,NaN,EMERGENCY,282599,2156-05-01 18:10:12,2156-05-03 18:43:45,2.0233,81.189041,98.6,0
4,249,NaN,158975,2156-04-27 15:33:00,2156-05-14 15:30:00,NaN,EMERGENCY,263882,2156-05-10 17:47:35,2156-05-11 19:16:03,1.0614,81.213699,314.2,0


In [45]:
cohorte_test_benchmark = pd.read_csv('/data/codi/OASIS/lengthOfStay/test/listfile.csv')
print(cohorte_test_benchmark.shape)
cohorte_test_benchmark.head()

(5282, 1)


,ICUSTAY_ID
0,247610
1,232110
2,277998
3,228977
4,277021


In [46]:
test=pd.merge(cohorte_test_benchmark, cohorte_general, how='left', on=['ICUSTAY_ID'])
print(test.shape) #asegurar que estan todos los icustays de cohorte_test_benchmark en cohorte general
test.head()

(5282, 14)


,ICUSTAY_ID,SUBJECT_ID,DOD,HADM_ID,ADMITTIME,DISCHTIME,DEATHTIME,ADMISSION_TYPE,INTIME,OUTTIME,LOS,AGE,PRELOS,MORTALITY_INHOSPITAL
0,247610,10000,NaN,187813,2186-08-10 21:57:00,2186-08-31 17:00:00,NaN,EMERGENCY,2186-08-14 10:50:09,2186-08-15 18:40:05,1.3263,49.704110,84.9,0
1,232110,10011,2126-08-28,105331,2126-08-14 22:32:00,2126-08-28 18:59:00,2126-08-28 18:59:00,EMERGENCY,2126-08-14 22:34:00,2126-08-28 18:59:00,13.8507,36.213699,0.0,1
2,277998,10012,NaN,122132,2112-09-19 07:15:00,2112-09-23 14:25:00,NaN,ELECTIVE,2112-09-19 10:22:56,2112-09-20 20:16:39,1.4123,69.610959,3.1,0
3,228977,10019,2163-05-15,177759,2163-05-14 20:43:00,2163-05-15 12:00:00,2163-05-15 12:00:00,EMERGENCY,2163-05-14 20:43:56,2163-05-16 03:47:04,1.2938,48.931507,0.0,1
4,277021,10026,2195-11-24,103770,2195-05-17 07:39:00,2195-05-24 11:45:00,NaN,EMERGENCY,2195-05-17 07:40:18,2195-05-19 17:24:25,2.4056,96.967177,0.0,0


In [47]:
test.isnull().sum() 
#asegurar que estan todos los icustays de cohorte_test_benchmark en cohorte general

ICUSTAY_ID                 0
SUBJECT_ID                 0
DOD                     3075
HADM_ID                    0
ADMITTIME                  0
DISCHTIME                  0
DEATHTIME               4792
ADMISSION_TYPE             0
INTIME                     0
OUTTIME                    0
LOS                        0
AGE                        0
PRELOS                     0
MORTALITY_INHOSPITAL       0
dtype: int64

In [48]:
#Solo necesito iscustay_id (ya que en este version nombré las carpetas con icustay_id)
test = pd.DataFrame(test['ICUSTAY_ID'])
test.head()

,ICUSTAY_ID
0,247610
1,232110
2,277998
3,228977
4,277021


In [49]:
output_resources_path = '/data/codi/OASIS/timesSeriesv2_cohorte/data/lengthOfStay'
try:
    os.makedirs(output_resources_path)
except:
    pass


In [50]:
test.to_csv(os.path.join(output_resources_path, 'testset.csv'),index=False,header=False)

In [51]:
cohorte_train_benchmark = pd.read_csv('/data/codi/OASIS/lengthOfStay/train/listfile.csv')
print(cohorte_train_benchmark.shape)
cohorte_train_benchmark.head()

(29846, 1)


,ICUSTAY_ID
0,209804
1,208696
2,229628
3,263325
4,269666


In [52]:
trainset=pd.merge(cohorte_train_benchmark, cohorte_general, how='left', on=['ICUSTAY_ID'])
print(trainset.shape)
trainset.head()

(29846, 14)


,ICUSTAY_ID,SUBJECT_ID,DOD,HADM_ID,ADMITTIME,DISCHTIME,DEATHTIME,ADMISSION_TYPE,INTIME,OUTTIME,LOS,AGE,PRELOS,MORTALITY_INHOSPITAL
0,209804,91051,NaN,195649,2162-11-24 19:45:00,2162-11-29 17:22:00,NaN,EMERGENCY,2162-11-24 22:32:55,2162-11-26 19:03:24,1.8545,43.531507,2.8,0
1,208696,19208,2140-01-15,138876,2139-10-15 23:18:00,2139-10-22 17:00:00,NaN,EMERGENCY,2139-10-17 08:45:08,2139-10-18 15:51:31,1.2961,80.747945,33.5,0
2,229628,41432,2149-01-20,166813,2148-01-23 21:14:00,2148-01-26 21:00:00,NaN,EMERGENCY,2148-01-23 21:15:35,2148-01-25 18:49:58,1.8989,86.813699,0.0,0
3,263325,65089,NaN,183040,2130-12-11 20:09:00,2130-12-17 11:00:00,NaN,EMERGENCY,2130-12-11 20:10:55,2130-12-13 17:25:26,1.8851,69.873973,0.0,0
4,269666,17882,NaN,151927,2148-01-15 11:52:00,2148-01-22 18:11:00,NaN,EMERGENCY,2148-01-15 11:53:14,2148-01-17 18:19:57,2.2686,64.986301,0.0,0


In [53]:
trainset.isnull().sum() 

ICUSTAY_ID                  0
SUBJECT_ID                  0
DOD                     16809
HADM_ID                     0
ADMITTIME                   0
DISCHTIME                   0
DEATHTIME               26724
ADMISSION_TYPE              0
INTIME                      0
OUTTIME                     0
LOS                         0
AGE                         0
PRELOS                      0
MORTALITY_INHOSPITAL        0
dtype: int64

In [54]:
trainset = pd.DataFrame(trainset['ICUSTAY_ID'])
trainset.head()

,ICUSTAY_ID
0,209804
1,208696
2,229628
3,263325
4,269666


In [55]:
trainset.to_csv(os.path.join(output_resources_path, 'trainset.csv'),index=False,header=False)

In [56]:
def process_partition(root_path, output_path, partition, listfile_path, eps=1e-6, n_hours=24):
    output_dir = os.path.join(output_path, partition)
    if not os.path.exists(output_dir):
        os.mkdir(output_dir)

    xy_pairs = []

    #Iterar los icustays con mismo orden que listfile de otros modelos. (para luego el cross validation sea sobre mismos splits)
    partition_set = []
    with open(listfile_path, "r") as partition_set_file:
        for line in partition_set_file:
            x = line.replace("\n", "")
            partition_set.append(x)
    

    #for icustay in tqdm(['201645','239511','212355'], desc='Iterating over stays in {}'.format(partition)): #prueba

    for icustay in tqdm(partition_set, desc='Iterating over stays in {}'.format(partition)):
        icustay_folder = os.path.join(root_path, icustay)
        icustay_ts_files = list(filter(lambda x: x.find("timeseries") != -1, os.listdir(icustay_folder)))

        for ts_filename in icustay_ts_files:
            with open(os.path.join(icustay_folder, ts_filename)) as tsfile:
                lb_filename = ts_filename.replace("_timeseries", "")
                label_df = pd.read_csv(os.path.join(icustay_folder, lb_filename))

                # empty label file
                if label_df.shape[0] == 0:
                    continue

                    
                if (label_df.iloc[0]['Length of Stay'] > 3):
                    los_label = 1
                else:
                    los_label = 0
                
                los = 24.0 * label_df.iloc[0]['Length of Stay']  # in hours
                if pd.isnull(los):
                    print("\n\t(length of stay is missing)", icustay, ts_filename)
                    continue

                #Esto ya esta controlado si usa listfile
                #if los < n_hours - eps:
                    #print("\n\t(length of stay < 24h)", icustay, ts_filename)
                    #continue

                ts_lines_conheader = tsfile.readlines()
                header = ts_lines_conheader[0]
                ts_lines = ts_lines_conheader[1:]
                event_times = [float(line.split(',')[0]) for line in ts_lines]

                ts_lines = [line for (line, t) in zip(ts_lines, event_times)
                            if -eps < t < (n_hours - 1.0 + eps)] #las horas cuentan de 0 a 23.
 
                # no measurements in ICU
                #En aqui no los excluyo porque quiero tener mismo cohorte que benchmark
                if len(ts_lines) == 0:
                    
                    splits = ts_lines_conheader[1].split(',')
                    splits[0] = '0.0'
                    for i in range(1, 9):
                        splits[i]=''
                    
                    line_age_Prelos_hour0 = ','
                    line_age_Prelos_hour0 = line_age_Prelos_hour0.join(splits)
                    #line_age_Prelos_hour0 se queda como '0.0,'+''+','+''+','+''+','+''+','+''+','+''+','+''+','+''+','+age+','+prelos+'\n'
                    #print(line_age_Prelos_hour0)
                    ts_lines = [line_age_Prelos_hour0]                    
                    print("\n\t(no events except age and prelos in first hours in ICU) ", icustay, ts_filename)
                    #continue
                    
                output_ts_filename = icustay + "_" + ts_filename
                with open(os.path.join(output_dir, output_ts_filename), "w") as outfile:
                    outfile.write(header)
                    for line in ts_lines:
                        outfile.write(line)

                xy_pairs.append((output_ts_filename, los_label))

    print("Number of created samples:", len(xy_pairs))

    with open(os.path.join(output_dir, "listfile.csv"), "w") as listfile:
        listfile.write('stay,y_true\n')
        for (x, y) in xy_pairs:
            listfile.write('{},{:d}\n'.format(x, y))




In [57]:
root_path = '/data/codi/OASIS/timesSeriesv2_cohorte/data/root/'
output_path = '/data/codi/OASIS/timesSeriesv2_cohorte/data/lengthOfStay'
testset_listfile_path = '/data/codi/OASIS/timesSeriesv2_cohorte/data/lengthOfStay/testset.csv'
trainset_listfile_path = '/data/codi/OASIS/timesSeriesv2_cohorte/data/lengthOfStay/trainset.csv'

In [58]:
if not os.path.exists(output_path):
    os.makedirs(output_path)

process_partition(root_path,output_path, "test", testset_listfile_path)
process_partition(root_path,output_path, "train", trainset_listfile_path)

Iterating over stays in test:   3%|▏     | 175/5282 [00:02<01:08, 74.38it/s]


	(no events except age and prelos in first hours in ICU)  239511 episode1_timeseries.csv


Iterating over stays in test:   5%|▎     | 270/5282 [00:04<00:58, 85.36it/s]


	(no events except age and prelos in first hours in ICU)  203388 episode1_timeseries.csv


Iterating over stays in test:   9%|▌     | 487/5282 [00:07<01:54, 41.87it/s]

...


	(no events except age and prelos in first hours in ICU)  251459 episode1_timeseries.csv


Iterating over stays in train:  99%|█▉| 29608/29846 [16:07<00:06, 34.48it/s]


	(no events except age and prelos in first hours in ICU)  224542 episode1_timeseries.csv


Iterating over stays in train: 100%|██| 29846/29846 [16:13<00:00, 30.67it/s]


Number of created samples: 29846


In [59]:
#pd.read_csv('/home/usuario/TFG/code/timesSeriesv2_cohorte/data/lengthOfStay/test/295741_episode1_timeseries.csv')

In [60]:
pd.read_csv('/data/codi/OASIS/timesSeriesv2_cohorte/data/lengthOfStay/test/239511_episode1_timeseries.csv')

,Hours,HRATE,MAP,RESP_RATE,TEMP_C,gcs_e,gcs_m,gcs_total_carevue,gcs_v,AGE,PRELOS
0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,63.139726,0.0


In [61]:
pd.read_csv('/data/codi/OASIS/timesSeriesv2_cohorte/data/lengthOfStay/train/201645_episode1_timeseries.csv')

,Hours,HRATE,MAP,RESP_RATE,TEMP_C,gcs_e,gcs_m,gcs_total_carevue,gcs_v,AGE,PRELOS
0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,67.241096,114.6


#### Create a unique timeSeries dataset in csv format for prediction task.

In [62]:
stays_root_path = '/data/codi/OASIS/timesSeriesv2_cohorte/data/lengthOfStay/train'

In [63]:
result_OneBigDataset = pd.DataFrame()
y_true =  pd.DataFrame()
listfile = pd.read_csv(os.path.join(stays_root_path,'listfile.csv'))

#normal range of measuraments
values = {'gcs_e': 4, 'gcs_m': 6, 'gcs_v': 5, 'gcs_total_carevue': 15,
                      'HRATE': 86,'MAP':77,'RESP_RATE':19,'TEMP_C':36.6}
            

#Iterar los icustays con mismo orden que listfile de otros modelos. (para luego el cross validation sea sobre mismos splits)
lista_stays_en_orden = listfile['stay'].tolist()

#for stay_dir in tqdm(['201645_episode1_timeseries.csv'], desc='Iterating over stays'): #prueba
    
for stay_dir in tqdm(lista_stays_en_orden, desc='Iterating over stays'):
    dn = os.path.join(stays_root_path, stay_dir)
        
    try:
        # reading timeseries and concate
        stayTimeSeries = pd.read_csv(dn)
        stayTimeSeries['Hours'] = np.round(stayTimeSeries['Hours'],0) 
        stayTimeSeries = stayTimeSeries.drop_duplicates(subset=['Hours'], keep='last')
        new_index = pd.Index(np.arange(0,24,1), name="Hours")
        stayTimeSeries = stayTimeSeries.set_index("Hours").reindex(new_index).reset_index()
        
        stayTimeSeries.fillna(method='ffill', inplace=True)
        #Dummy solution if whole column is nan or no have previous non nan value, fill nan with normal range
        #Para Age y Prelos seguro tiene algun valor
        stayTimeSeries['AGE'].fillna(method='bfill', inplace=True)
        stayTimeSeries['PRELOS'].fillna(method='bfill', inplace=True)

        if(stayTimeSeries['AGE'].isnull().all() or stayTimeSeries['PRELOS'].isnull().all()):
            print('Error reading age and prelos for icustay_id: {}\n'.format(stay_dir))
            #continue

        stayTimeSeries.fillna(value=values,inplace=True)
        result_OneBigDataset = pd.concat([result_OneBigDataset,stayTimeSeries])

        stay_y_true =  listfile.loc[listfile['stay'] == stay_dir]['y_true']
        y_true = pd.concat([y_true,stay_y_true], ignore_index=True)

    except:
        sys.stderr.write('Error reading from disk for icustay_id: {}\n'.format(stay_dir))
        continue



Iterating over stays: 100%|███████████| 29846/29846 [13:02<00:00, 38.13it/s]


In [64]:
#prueba_train = np.array(result_OneBigDataset.drop(['Hours'],axis=1)).reshape(y_true.shape[0], 24, 10)
#prueba_train.shape

In [65]:
result_OneBigDataset.head(50)

,Hours,HRATE,MAP,RESP_RATE,TEMP_C,gcs_e,gcs_m,gcs_total_carevue,gcs_v,AGE,PRELOS
0,0,97.0,74.000000,23.0,36.600000,4.0,6.0,15.0,5.0,43.531507,2.8
1,1,96.0,74.000000,22.0,36.600000,3.0,6.0,15.0,1.0,43.531507,2.8
2,2,90.0,74.000000,20.0,36.600000,3.0,6.0,15.0,1.0,43.531507,2.8
3,3,90.0,76.000000,16.0,36.600000,3.0,6.0,15.0,1.0,43.531507,2.8
4,4,81.0,76.000000,16.0,36.600000,3.0,6.0,15.0,1.0,43.531507,2.8
5,5,80.0,76.000000,15.0,36.944444,3.0,6.0,15.0,1.0,43.531507,2.8
6,6,83.0,81.000000,18.0,36.944444,3.0,6.0,15.0,1.0,43.531507,2.8
7,7,88.0,81.000000,18.0,36.944444,3.0,6.0,15.0,1.0,43.531507,2.8
8,8,86.0,81.000000,16.0,36.944444,3.0,6.0,15.0,1.0,43.531507,2.8
9,9,89.0,78.000000,18.0,37.055556,3.0,6.0,15.0,5.0,43.531507,2.8


In [66]:
result_OneBigDataset.to_csv('/data/codi/OASIS/timesSeriesv2_cohorte/data/lengthOfStay/result_OneBigDataset.csv',index=False)
y_true.to_csv('/data/codi/OASIS/timesSeriesv2_cohorte/data/lengthOfStay/result_OneBigDataset_y_true.csv',index=False, header=['y_true'])

In [67]:
result_OneBigDataset = pd.read_csv('/data/codi/OASIS/timesSeriesv2_cohorte/data/lengthOfStay/result_OneBigDataset.csv')
y_true = pd.read_csv('/data/codi/OASIS/timesSeriesv2_cohorte/data/lengthOfStay/result_OneBigDataset_y_true.csv')

In [68]:
result_OneBigDataset.isnull().sum()

Hours                0
HRATE                0
MAP                  0
RESP_RATE            0
TEMP_C               0
gcs_e                0
gcs_m                0
gcs_total_carevue    0
gcs_v                0
AGE                  0
PRELOS               0
dtype: int64

In [69]:
stays_root_path = '/data/codi/OASIS/timesSeriesv2_cohorte/data/lengthOfStay/test'

In [70]:
result_OneBigDataset_test = pd.DataFrame()
y_true =  pd.DataFrame()
listfile = pd.read_csv(os.path.join(stays_root_path,'listfile.csv'))


#normal range of measuraments
values = {'gcs_e': 4, 'gcs_m': 6, 'gcs_v': 5, 'gcs_total_carevue': 15,
                      'HRATE': 86,'MAP':77,'RESP_RATE':19,'TEMP_C':36.6}

#Iterar los icustays con mismo orden que listfile de otros modelos. (para luego el cross validation sea sobre mismos splits)
lista_stays_en_orden = listfile['stay'].tolist()

for stay_dir in tqdm(lista_stays_en_orden, desc='Iterating over stays'):
    dn = os.path.join(stays_root_path, stay_dir)
        
    try:
        # reading timeseries and concate
        stayTimeSeries = pd.read_csv(dn)
        #manera 2: age y prelos para cada icustay id es estático:
        #age = stayTimeSeries['AGE'][0]
        #prelos = stayTimeSeries['PRELOS'][0]
        stayTimeSeries['Hours'] = np.round(stayTimeSeries['Hours'],0)
        stayTimeSeries = stayTimeSeries.drop_duplicates(subset=['Hours'], keep='last')
        new_index = pd.Index(np.arange(0,24,1), name="Hours")
        stayTimeSeries = stayTimeSeries.set_index("Hours").reindex(new_index).reset_index()


        stayTimeSeries.fillna(method='ffill', inplace=True)
        #Dummy solution if whole column is nan or no have previous non nan value, fill nan with normal range

        #Para Age y Prelos seguro tiene algun valor
        stayTimeSeries['AGE'].fillna(method='bfill', inplace=True)
        stayTimeSeries['PRELOS'].fillna(method='bfill', inplace=True)

        #manera 2:Para Age y Prelos seguro tiene algun valor
        #stayTimeSeries['AGE'].fillna(value=age, inplace=True)
        #stayTimeSeries['PRELOS'].fillna(value=prelos, inplace=True)

        
        if(stayTimeSeries['AGE'].isnull().all() or stayTimeSeries['PRELOS'].isnull().all()):
            print('Error reading age and prelos for icustay_id: {}\n'.format(stay_dir))
            #continue

        stayTimeSeries.fillna(value=values,inplace=True)
        result_OneBigDataset_test = pd.concat([result_OneBigDataset_test,stayTimeSeries])

        stay_y_true =  listfile.loc[listfile['stay'] == stay_dir]['y_true']
        y_true = pd.concat([y_true,stay_y_true], ignore_index=True)

    except:
        sys.stderr.write('Error reading from disk for icustay_id: {}\n'.format(stay_dir))
        continue



Iterating over stays: 100%|█████████████| 5282/5282 [01:17<00:00, 67.98it/s]


In [71]:
#Comprobar que son de mismo tamaño que benchmark 5282
prueba_test = np.array(result_OneBigDataset_test.drop(['Hours'],axis=1)).reshape(y_true.shape[0], 24, 10)
prueba_test.shape

(5282, 24, 10)

In [72]:
result_OneBigDataset_test.head(50)

,Hours,HRATE,MAP,RESP_RATE,TEMP_C,gcs_e,gcs_m,gcs_total_carevue,gcs_v,AGE,PRELOS
0,0,54.0,89.000000,14.0,36.600000,4.0,6.0,15.0,5.0,49.704110,84.9
1,1,54.0,77.000000,15.0,36.600000,4.0,6.0,15.0,5.0,49.704110,84.9
2,2,55.0,80.333298,10.0,36.111099,4.0,6.0,15.0,5.0,49.704110,84.9
3,3,57.0,69.333298,8.0,36.111099,4.0,6.0,15.0,5.0,49.704110,84.9
4,4,63.0,71.333298,14.0,35.777802,4.0,6.0,15.0,5.0,49.704110,84.9
5,5,63.0,70.666702,13.0,35.777802,4.0,6.0,15.0,5.0,49.704110,84.9
6,6,61.0,75.666702,9.0,35.777802,4.0,6.0,15.0,5.0,49.704110,84.9
7,7,58.0,66.000000,8.0,35.777802,4.0,6.0,15.0,5.0,49.704110,84.9
8,8,56.0,82.000000,8.0,35.944401,4.0,6.0,15.0,5.0,49.704110,84.9
9,9,61.0,70.666702,22.0,35.944401,4.0,6.0,15.0,5.0,49.704110,84.9


In [73]:
result_OneBigDataset_test.to_csv('/data/codi/OASIS/timesSeriesv2_cohorte/data/lengthOfStay/result_OneBigDataset_test.csv',index=False)
y_true.to_csv('/data/codi/OASIS/timesSeriesv2_cohorte/data/lengthOfStay/result_OneBigDataset_y_true_test.csv',index=False, header=['y_true'])

In [74]:
result_OneBigDataset_test = pd.read_csv('/data/codi/OASIS/timesSeriesv2_cohorte/data/lengthOfStay/result_OneBigDataset_test.csv')
y_true_test = pd.read_csv('/data/codi/OASIS/timesSeriesv2_cohorte/data/lengthOfStay/result_OneBigDataset_y_true_test.csv')

In [75]:
result_OneBigDataset_test.isnull().sum()

Hours                0
HRATE                0
MAP                  0
RESP_RATE            0
TEMP_C               0
gcs_e                0
gcs_m                0
gcs_total_carevue    0
gcs_v                0
AGE                  0
PRELOS               0
dtype: int64

### TimeSeries For Mortality task:

In [76]:
stays_root_path = "/data/codi/OASIS/timesSeriesv2_cohorte/data/root"

In [77]:
cohorte_general = pd.read_csv(os.path.join(stays_root_path, "all_stays.csv"))
cohorte_general.head()

,SUBJECT_ID,DOD,HADM_ID,ADMITTIME,DISCHTIME,DEATHTIME,ADMISSION_TYPE,ICUSTAY_ID,INTIME,OUTTIME,LOS,AGE,PRELOS,MORTALITY_INHOSPITAL
0,249,NaN,116935,2149-12-17 20:41:00,2149-12-31 14:55:00,NaN,EMERGENCY,215044,2149-12-18 20:06:02,2149-12-24 13:31:45,5.7262,74.816438,23.4,0
1,249,NaN,149546,2155-02-03 20:16:00,2155-02-14 11:15:00,NaN,EMERGENCY,269035,2155-02-03 20:17:29,2155-02-05 18:34:02,1.9282,79.947945,0.0,0
2,249,NaN,149546,2155-02-03 20:16:00,2155-02-14 11:15:00,NaN,EMERGENCY,263055,2155-02-07 18:51:16,2155-02-11 16:00:39,3.8815,79.958904,94.6,0
3,249,NaN,158975,2156-04-27 15:33:00,2156-05-14 15:30:00,NaN,EMERGENCY,282599,2156-05-01 18:10:12,2156-05-03 18:43:45,2.0233,81.189041,98.6,0
4,249,NaN,158975,2156-04-27 15:33:00,2156-05-14 15:30:00,NaN,EMERGENCY,263882,2156-05-10 17:47:35,2156-05-11 19:16:03,1.0614,81.213699,314.2,0


In [78]:
cohorte_test_benchmark = pd.read_csv('/data/codi/OASIS/mortality/test/listfile.csv')
print(cohorte_test_benchmark.shape)
cohorte_test_benchmark.head()

(3236, 1)


,ICUSTAY_ID
0,232110
1,277021
2,262782
3,258147
4,243600


In [79]:
test=pd.merge(cohorte_test_benchmark, cohorte_general, how='left', on=['ICUSTAY_ID'])
print(test.shape) #asegurar que estan todos los icustays de cohorte_test_benchmark en cohorte general
print(test.isnull().sum())
test.head()

(3236, 14)
ICUSTAY_ID                 0
SUBJECT_ID                 0
DOD                     1715
HADM_ID                    0
ADMITTIME                  0
DISCHTIME                  0
DEATHTIME               2873
ADMISSION_TYPE             0
INTIME                     0
OUTTIME                    0
LOS                        0
AGE                        0
PRELOS                     0
MORTALITY_INHOSPITAL       0
dtype: int64


,ICUSTAY_ID,SUBJECT_ID,DOD,HADM_ID,ADMITTIME,DISCHTIME,DEATHTIME,ADMISSION_TYPE,INTIME,OUTTIME,LOS,AGE,PRELOS,MORTALITY_INHOSPITAL
0,232110,10011,2126-08-28,105331,2126-08-14 22:32:00,2126-08-28 18:59:00,2126-08-28 18:59:00,EMERGENCY,2126-08-14 22:34:00,2126-08-28 18:59:00,13.8507,36.213699,0.0,1
1,277021,10026,2195-11-24,103770,2195-05-17 07:39:00,2195-05-24 11:45:00,NaN,EMERGENCY,2195-05-17 07:40:18,2195-05-19 17:24:25,2.4056,96.967177,0.0,0
2,262782,10030,NaN,124443,2137-08-21 14:17:00,2137-08-24 14:56:00,NaN,URGENT,2137-08-21 14:18:00,2137-08-23 15:17:00,2.0410,77.331507,0.0,0
3,258147,10042,2150-12-03,148562,2147-02-06 12:38:00,2147-02-17 19:00:00,NaN,EMERGENCY,2147-02-06 12:40:59,2147-02-14 16:04:43,8.1415,70.800000,0.0,0
4,243600,10094,2180-03-20,168074,2180-02-29 18:54:00,2180-03-10 17:35:00,NaN,EMERGENCY,2180-02-29 18:54:39,2180-03-04 21:20:36,4.1014,94.980058,0.0,0


In [80]:
#Solo necesito iscustay_id (ya que en este version nombré las carpetas con icustay_id)
test = pd.DataFrame(test['ICUSTAY_ID'])
test.head()


,ICUSTAY_ID
0,232110
1,277021
2,262782
3,258147
4,243600


In [81]:
output_resources_path = '/data/codi/OASIS/timesSeriesv2_cohorte/data/mortality'
try:
    os.makedirs(output_resources_path)
except:
    pass

In [82]:
test.to_csv(os.path.join(output_resources_path, 'testset.csv'),index=False,header=False)

In [83]:
cohorte_train_benchmark = pd.read_csv('/data/codi/OASIS/mortality/train/listfile.csv')
print(cohorte_train_benchmark.shape)
cohorte_train_benchmark.head()

(17903, 1)


,ICUSTAY_ID
0,219326
1,273321
2,221917
3,269490
4,225465


In [84]:
trainset=pd.merge(cohorte_train_benchmark, cohorte_general, how='left', on=['ICUSTAY_ID'])
print(trainset.shape)
print(trainset.isnull().sum())
trainset.head()

(17903, 14)
ICUSTAY_ID                  0
SUBJECT_ID                  0
DOD                      9080
HADM_ID                     0
ADMITTIME                   0
DISCHTIME                   0
DEATHTIME               15492
ADMISSION_TYPE              0
INTIME                      0
OUTTIME                     0
LOS                         0
AGE                         0
PRELOS                      0
MORTALITY_INHOSPITAL        0
dtype: int64


,ICUSTAY_ID,SUBJECT_ID,DOD,HADM_ID,ADMITTIME,DISCHTIME,DEATHTIME,ADMISSION_TYPE,INTIME,OUTTIME,LOS,AGE,PRELOS,MORTALITY_INHOSPITAL
0,219326,84984,2114-09-28,101295,2114-09-15 02:04:00,2114-09-28 07:45:00,2114-09-28 07:45:00,EMERGENCY,2114-09-15 02:04:54,2114-09-28 11:14:08,13.3814,77.334247,0.0,1
1,273321,57299,2125-09-08,189271,2122-11-30 09:00:00,2122-12-05 12:39:00,NaN,ELECTIVE,2122-11-30 14:51:39,2122-12-03 11:28:34,2.8590,80.717808,5.9,0
2,221917,72032,NaN,190953,2130-12-25 16:08:00,2131-01-02 14:30:00,NaN,EMERGENCY,2130-12-25 16:09:00,2130-12-31 18:15:36,6.0879,56.183562,0.0,0
3,269490,22175,NaN,129470,2122-10-30 12:55:00,2122-11-17 18:00:00,NaN,EMERGENCY,2122-10-30 21:47:49,2122-11-17 19:09:25,17.8900,64.424658,8.9,0
4,225465,13033,2188-02-10,151787,2183-07-23 23:17:00,2183-07-26 16:50:00,NaN,EMERGENCY,2183-07-23 23:19:39,2183-07-26 16:50:08,2.7295,35.271233,0.0,0


In [85]:
trainset = pd.DataFrame(trainset['ICUSTAY_ID'])
trainset.head()

,ICUSTAY_ID
0,219326
1,273321
2,221917
3,269490
4,225465


In [86]:
trainset.to_csv(os.path.join(output_resources_path, 'trainset.csv'),index=False,header=False)

In [87]:
def process_partition(root_path, output_path, partition, listfile_path, eps=1e-6, n_hours=24):
    output_dir = os.path.join(output_path, partition)
    if not os.path.exists(output_dir):
        os.mkdir(output_dir)

    xy_pairs = []

    #Iterar los icustays con mismo orden que listfile de otros modelos. (para luego el cross validation sea sobre mismos splits)
    partition_set = []
    with open(listfile_path, "r") as partition_set_file:
        for line in partition_set_file:
            x = line.replace("\n", "")
            partition_set.append(x)
    

    #for icustay in tqdm(['201645','239511','212355'], desc='Iterating over stays in {}'.format(partition)): #prueba

    for icustay in tqdm(partition_set, desc='Iterating over stays in {}'.format(partition)):
        icustay_folder = os.path.join(root_path, icustay)
        icustay_ts_files = list(filter(lambda x: x.find("timeseries") != -1, os.listdir(icustay_folder)))

        for ts_filename in icustay_ts_files:
            with open(os.path.join(icustay_folder, ts_filename)) as tsfile:
                lb_filename = ts_filename.replace("_timeseries", "")
                label_df = pd.read_csv(os.path.join(icustay_folder, lb_filename))

                # empty label file
                if label_df.shape[0] == 0:
                    continue

                    
                #1 positive = dead, 0 negative, survived
                mortality_label = label_df.iloc[0]['Inhospital Mortality']
                los = 24.0 * label_df.iloc[0]['Length of Stay']  # in hours
                if pd.isnull(los):
                    print("\n\t(length of stay is missing)", icustay, ts_filename)
                    continue

                #Esto ya esta controlado con listfile
                #if los < n_hours - eps:
                    #print("\n\t(length of stay < 24h)", icustay, ts_filename)
                    #continue

                ts_lines_conheader = tsfile.readlines()
                header = ts_lines_conheader[0]
                ts_lines = ts_lines_conheader[1:]
                event_times = [float(line.split(',')[0]) for line in ts_lines]

                ts_lines = [line for (line, t) in zip(ts_lines, event_times)
                            if -eps < t < (n_hours - 1.0 + eps)] #las horas cuentan de 0 a 23.
 
                # no measurements in ICU
                #En aqui no lo tengo en cuenta porque quiero tener mismo cohorte que benchmark
                if len(ts_lines) == 0:
                    
                    splits = ts_lines_conheader[1].split(',')
                    splits[0] = '0.0'
                    for i in range(1, 9):
                        splits[i]=''
                    
                    line_age_Prelos_hour0 = ','
                    line_age_Prelos_hour0 = line_age_Prelos_hour0.join(splits)
                    ts_lines = [line_age_Prelos_hour0]                    
                    print("\n\t(no events except age and prelos in first hours in ICU) ", icustay, ts_filename)
                    #continue
                    
                output_ts_filename = icustay + "_" + ts_filename
                with open(os.path.join(output_dir, output_ts_filename), "w") as outfile:
                    outfile.write(header)
                    for line in ts_lines:
                        outfile.write(line)

                xy_pairs.append((output_ts_filename, mortality_label))

    print("Number of created samples:", len(xy_pairs))

    with open(os.path.join(output_dir, "listfile.csv"), "w") as listfile:
        listfile.write('stay,y_true\n')
        for (x, y) in xy_pairs:
            listfile.write('{},{:d}\n'.format(x, y))


In [88]:
root_path = '/data/codi/OASIS/timesSeriesv2_cohorte/data/root/'
output_path = '/data/codi/OASIS/timesSeriesv2_cohorte/data/mortality'
testset_listfile_path = '/data/codi/OASIS/timesSeriesv2_cohorte/data/mortality/testset.csv'
trainset_listfile_path = '/data/codi/OASIS/timesSeriesv2_cohorte/data/mortality/trainset.csv'

In [89]:
if not os.path.exists(output_path):
    os.makedirs(output_path)

process_partition(root_path,output_path, "test", testset_listfile_path)
process_partition(root_path,output_path, "train", trainset_listfile_path)

Iterating over stays in test:   4%|▏    | 133/3236 [00:01<00:25, 124.11it/s]


	(no events except age and prelos in first hours in ICU)  239511 episode1_timeseries.csv


Iterating over stays in test:   6%|▎    | 185/3236 [00:01<00:25, 121.98it/s]


	(no events except age and prelos in first hours in ICU)  203388 episode1_timeseries.csv


Iterating over stays in test:  10%|▍    | 320/3236 [00:02<00:26, 109.77it/s]

...


	(no events except age and prelos in first hours in ICU)  276557 episode1_timeseries.csv

	(no events except age and prelos in first hours in ICU)  224922 episode1_timeseries.csv


Iterating over stays in train:  99%|█▉| 17728/17903 [03:34<00:01, 99.95it/s]


	(no events except age and prelos in first hours in ICU)  290213 episode1_timeseries.csv


Iterating over stays in train: 100%|██| 17903/17903 [03:36<00:00, 82.73it/s]


Number of created samples: 17903


#### Create a unique timeSeries dataset in csv format for prediction task.

In [90]:
stays_root_path = '/data/codi/OASIS/timesSeriesv2_cohorte/data/mortality/train'

In [91]:
result_OneBigDataset = pd.DataFrame()
y_true =  pd.DataFrame()
listfile = pd.read_csv(os.path.join(stays_root_path,'listfile.csv'))

#normal range of measuraments
values = {'gcs_e': 4, 'gcs_m': 6, 'gcs_v': 5, 'gcs_total_carevue': 15,
                      'HRATE': 86,'MAP':77,'RESP_RATE':19,'TEMP_C':36.6}
            

#Iterar los icustays con mismo orden que listfile de otros modelos. (para luego el cross validation sea sobre mismos splits)
lista_stays_en_orden = listfile['stay'].tolist()

#for stay_dir in tqdm(['201645_episode1_timeseries.csv'], desc='Iterating over stays'): #prueba
    
for stay_dir in tqdm(lista_stays_en_orden, desc='Iterating over stays'):
    dn = os.path.join(stays_root_path, stay_dir)
        
    try:
        # reading timeseries and concate
        stayTimeSeries = pd.read_csv(dn)
        stayTimeSeries['Hours'] = np.round(stayTimeSeries['Hours'],0) 
        stayTimeSeries = stayTimeSeries.drop_duplicates(subset=['Hours'], keep='last')
        new_index = pd.Index(np.arange(0,24,1), name="Hours")
        stayTimeSeries = stayTimeSeries.set_index("Hours").reindex(new_index).reset_index()
        
        stayTimeSeries.fillna(method='ffill', inplace=True)
        #Dummy solution if whole column is nan or no have previous non nan value, fill nan with normal range
        #Para Age y Prelos seguro tiene algun valor
        stayTimeSeries['AGE'].fillna(method='bfill', inplace=True)
        stayTimeSeries['PRELOS'].fillna(method='bfill', inplace=True)

        if(stayTimeSeries['AGE'].isnull().all() or stayTimeSeries['PRELOS'].isnull().all()):
            print('Error reading age and prelos for icustay_id: {}\n'.format(stay_dir))
            #continue

        stayTimeSeries.fillna(value=values,inplace=True)
        result_OneBigDataset = pd.concat([result_OneBigDataset,stayTimeSeries])

        stay_y_true =  listfile.loc[listfile['stay'] == stay_dir]['y_true']
        y_true = pd.concat([y_true,stay_y_true], ignore_index=True)

    except:
        sys.stderr.write('Error reading from disk for icustay_id: {}\n'.format(stay_dir))
        continue



Iterating over stays: 100%|███████████| 17903/17903 [06:16<00:00, 47.61it/s]


In [92]:
result_OneBigDataset.to_csv('/data/codi/OASIS/timesSeriesv2_cohorte/data/mortality/result_OneBigDataset.csv',index=False)
y_true.to_csv('/data/codi/OASIS/timesSeriesv2_cohorte/data/mortality/result_OneBigDataset_y_true.csv',index=False, header=['y_true'])

In [93]:
result_OneBigDataset = pd.read_csv('/data/codi/OASIS/timesSeriesv2_cohorte/data/mortality/result_OneBigDataset.csv')
y_true = pd.read_csv('/data/codi/OASIS/timesSeriesv2_cohorte/data/mortality/result_OneBigDataset_y_true.csv')

In [94]:
prueba_train = np.array(result_OneBigDataset.drop(['Hours'],axis=1)).reshape(y_true.shape[0], 24, 10)
prueba_train.shape

(17903, 24, 10)

In [95]:
result_OneBigDataset.head(50)

,Hours,HRATE,MAP,RESP_RATE,TEMP_C,gcs_e,gcs_m,gcs_total_carevue,gcs_v,AGE,PRELOS
0,0,86.0,77.0,19.0,36.600000,4.0,6.0,15.0,5.0,77.334247,0.0
1,1,86.0,77.0,19.0,36.600000,4.0,6.0,15.0,5.0,77.334247,0.0
2,2,86.0,77.0,19.0,36.600000,4.0,6.0,15.0,5.0,77.334247,0.0
3,3,80.0,76.0,29.0,36.600000,4.0,6.0,15.0,5.0,77.334247,0.0
4,4,80.0,76.0,29.0,36.600000,4.0,6.0,15.0,5.0,77.334247,0.0
5,5,68.0,64.0,26.0,36.600000,4.0,6.0,15.0,5.0,77.334247,0.0
6,6,100.0,61.0,25.0,36.111111,4.0,6.0,15.0,5.0,77.334247,0.0
7,7,74.0,69.0,29.0,36.111111,4.0,6.0,15.0,5.0,77.334247,0.0
8,8,77.0,67.0,27.0,35.444444,4.0,6.0,15.0,5.0,77.334247,0.0
9,9,84.0,71.0,28.0,35.444444,4.0,6.0,15.0,5.0,77.334247,0.0


In [96]:
result_OneBigDataset.isnull().sum()

Hours                0
HRATE                0
MAP                  0
RESP_RATE            0
TEMP_C               0
gcs_e                0
gcs_m                0
gcs_total_carevue    0
gcs_v                0
AGE                  0
PRELOS               0
dtype: int64

In [97]:
stays_root_path = '/data/codi/OASIS/timesSeriesv2_cohorte/data/mortality/test'

In [98]:
result_OneBigDataset_test = pd.DataFrame()
y_true =  pd.DataFrame()
listfile = pd.read_csv(os.path.join(stays_root_path,'listfile.csv'))


#normal range of measuraments
values = {'gcs_e': 4, 'gcs_m': 6, 'gcs_v': 5, 'gcs_total_carevue': 15,
                      'HRATE': 86,'MAP':77,'RESP_RATE':19,'TEMP_C':36.6}

#Iterar los icustays con mismo orden que listfile de otros modelos. (para luego el cross validation sea sobre mismos splits)
lista_stays_en_orden = listfile['stay'].tolist()

for stay_dir in tqdm(lista_stays_en_orden, desc='Iterating over stays'):
    dn = os.path.join(stays_root_path, stay_dir)
        
    try:
        # reading timeseries and concate
        stayTimeSeries = pd.read_csv(dn)
        #manera 2: age y prelos para cada icustay id es estático:
        #age = stayTimeSeries['AGE'][0]
        #prelos = stayTimeSeries['PRELOS'][0]
        stayTimeSeries['Hours'] = np.round(stayTimeSeries['Hours'],0)
        stayTimeSeries = stayTimeSeries.drop_duplicates(subset=['Hours'], keep='last')
        new_index = pd.Index(np.arange(0,24,1), name="Hours")
        stayTimeSeries = stayTimeSeries.set_index("Hours").reindex(new_index).reset_index()


        stayTimeSeries.fillna(method='ffill', inplace=True)
        #Dummy solution if whole column is nan or no have previous non nan value, fill nan with normal range

        #Para Age y Prelos seguro tiene algun valor
        stayTimeSeries['AGE'].fillna(method='bfill', inplace=True)
        stayTimeSeries['PRELOS'].fillna(method='bfill', inplace=True)

        #manera 2:Para Age y Prelos seguro tiene algun valor
        #stayTimeSeries['AGE'].fillna(value=age, inplace=True)
        #stayTimeSeries['PRELOS'].fillna(value=prelos, inplace=True)

        
        if(stayTimeSeries['AGE'].isnull().all() or stayTimeSeries['PRELOS'].isnull().all()):
            print('Error reading age and prelos for icustay_id: {}\n'.format(stay_dir))
            #continue

        stayTimeSeries.fillna(value=values,inplace=True)
        result_OneBigDataset_test = pd.concat([result_OneBigDataset_test,stayTimeSeries])

        stay_y_true =  listfile.loc[listfile['stay'] == stay_dir]['y_true']
        y_true = pd.concat([y_true,stay_y_true], ignore_index=True)

    except:
        sys.stderr.write('Error reading from disk for icustay_id: {}\n'.format(stay_dir))
        continue


Iterating over stays: 100%|█████████████| 3236/3236 [00:45<00:00, 71.24it/s]


In [99]:
result_OneBigDataset_test.head(50)

,Hours,HRATE,MAP,RESP_RATE,TEMP_C,gcs_e,gcs_m,gcs_total_carevue,gcs_v,AGE,PRELOS
0,0,81.0,67.000000,23.0,36.600000,4.0,6.0,15.0,5.0,36.213699,0.0
1,1,76.0,62.666698,21.0,36.333302,4.0,6.0,15.0,5.0,36.213699,0.0
2,2,79.0,65.666702,20.0,36.333302,4.0,6.0,15.0,5.0,36.213699,0.0
3,3,72.0,65.000000,20.0,36.222198,4.0,6.0,15.0,5.0,36.213699,0.0
4,4,74.0,71.000000,21.0,36.222198,4.0,6.0,15.0,5.0,36.213699,0.0
5,5,78.0,67.333298,20.0,36.388901,4.0,6.0,15.0,5.0,36.213699,0.0
6,6,84.0,64.000000,21.0,36.388901,4.0,6.0,15.0,5.0,36.213699,0.0
7,7,95.0,65.666702,20.0,36.388901,4.0,6.0,15.0,5.0,36.213699,0.0
8,8,94.0,62.000000,19.0,37.055599,4.0,6.0,14.0,4.0,36.213699,0.0
9,9,108.0,75.000000,21.0,37.055599,4.0,6.0,14.0,4.0,36.213699,0.0


In [100]:
prueba_test = np.array(result_OneBigDataset_test.drop(['Hours'],axis=1)).reshape(y_true.shape[0], 24, 10)
prueba_test.shape

(3236, 24, 10)

In [101]:
result_OneBigDataset_test.to_csv('/data/codi/OASIS/timesSeriesv2_cohorte/data/mortality/result_OneBigDataset_test.csv',index=False)
y_true.to_csv('/data/codi/OASIS/timesSeriesv2_cohorte/data/mortality/result_OneBigDataset_y_true_test.csv',index=False, header=['y_true'])

In [102]:
result_OneBigDataset_test = pd.read_csv('/data/codi/OASIS/timesSeriesv2_cohorte/data/mortality/result_OneBigDataset_test.csv')
y_true_test = pd.read_csv('/data/codi/OASIS/timesSeriesv2_cohorte/data/mortality/result_OneBigDataset_y_true_test.csv')

In [103]:
result_OneBigDataset_test.isnull().sum()

Hours                0
HRATE                0
MAP                  0
RESP_RATE            0
TEMP_C               0
gcs_e                0
gcs_m                0
gcs_total_carevue    0
gcs_v                0
AGE                  0
PRELOS               0
dtype: int64